# Procesamiento de grafos en Spark con GraphFrames

Recordemos que GraphFrames no es parte de Spark. Es un paquete externo que permite procesar grafos usando Spark DataFrames en lugar de los RDD menos cómodos, como hacía el antiguo módulo GraphX que sí es parte de Spark. Se espera que GraphFrames se incorpore oficialmente a Spark en el futuro cercano.

Mientras tanto, primero debemos instalar el paquete GraphFrames para python, que es solo un wrapper de Python para el código Scala que contiene la implementación realmente paralela y distribuida. Podemos instalar el paquete de Python a través de `pip3 install`. Para la implementación de Scala, tenemos que indicar el paquete GraphFrames como "Spark package" que queremos usar en la opción de configuración `--packages` antes de crear el objeto SparkSession. Esto descargará la implementación GraphFrames para Scala sobre la marcha. El paquete de Python se basa por completo en dicho código Scala y generará un error si no se encuentra. Para que los notebooks de JupyterLab ya lleven esto incorporado, se añadió la propiedad de `spark.jars.packages` al crear el cluster en Dataproc. 

### Descargamos el fichero csv de vuelos y lo subimos a HDFS

In [1]:
!wget https://raw.githubusercontent.com/olbapjose/xapi-clojure/master/flights_jan08.csv
!hdfs dfs -copyFromLocal flights_jan08.csv /tmp

--2020-03-26 09:44:33--  https://raw.githubusercontent.com/olbapjose/xapi-clojure/master/flights_jan08.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9719583 (9.3M) [text/plain]
Saving to: ‘flights_jan08.csv’

flights_jan08.csv   100%[===================>]   9.27M  --.-KB/s    in 0.1s    

2020-03-26 09:44:34 (71.6 MB/s) - ‘flights_jan08.csv’ saved [9719583/9719583]



### Descargamos el paquete graphframes de python con el comando pip habitual de Python

In [3]:
!pip install graphframes

You are using pip version 9.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
from pyspark.sql import functions as F
from graphframes import GraphFrame

flightsDF = spark.read.option("header", "true").option("inferSchema", "true").csv("/tmp/flights_jan08.csv")
verticesDF = flightsDF.select(F.col("Origin").alias("id")).distinct().cache()
edgesDF = flightsDF.withColumnRenamed("Origin", "src")\
                   .withColumnRenamed("Dest", "dst")\
                   .select("src", "dst", "Distance")\
                   .distinct()\
                   .cache() # select a few columns just to keep things simple
                
graph = GraphFrame(verticesDF, edgesDF)

Vamos a calcular los aeropuertos con más vuelos como aquellos con el grado más alto en su vértice (suma de ambos).
Es sorprendente que el aeropuerto LAS (Las Vegas - McCarran International Airport) quede en primer lugar en número de vuelos!

In [6]:
graph.degrees.orderBy(F.col("degree").desc()).show(3)

+---+------+
| id|degree|
+---+------+
|LAS|   108|
|MDW|    94|
|PHX|    84|
+---+------+
only showing top 3 rows



In [7]:
graph.inDegrees.orderBy(F.col("inDegree").desc()).show(3)

+---+--------+
| id|inDegree|
+---+--------+
|LAS|      54|
|MDW|      47|
|PHX|      42|
+---+--------+
only showing top 3 rows



In [8]:
graph.outDegrees.orderBy(F.col("outDegree").desc()).show(3)

+---+---------+
| id|outDegree|
+---+---------+
|LAS|       54|
|MDW|       47|
|PHX|       42|
+---+---------+
only showing top 3 rows



## PageRank para determinar la importancia global de los vértices

Si ejecutamos PageRank, que es un poco más sofisticado que contar simplemente el número de aristas que llegan o salen de cada vértice, confirma que LAS tiene importancia estratégica en nuestro grafo en base a las conexiones con otros aeropuertos.

In [7]:
# ADVERTENCIA: pageRank tarda unos minutos en ejecutarse. Descomentar la siguiente línea si realmente queremos ejecutarlo :-)

#ranks = graph.pageRank(resetProbability=0.15, maxIter=10)
ranks.vertices.orderBy(F.col("pagerank").desc()).show()

+---+------------------+
| id|          pagerank|
+---+------------------+
|LAS|3.9681005880868927|
|MDW| 3.464188596739582|
|PHX|2.9996296419306288|
|BWI| 2.859044345021574|
|MCO| 2.506806330762528|
|TPA| 2.325980728922274|
|ABQ|  2.28062606472766|
|HOU| 2.242821989285586|
|BNA|2.1393487543945984|
|SAN|  2.10337109753917|
|MCI|1.9976511399324188|
|AUS|1.8642893476111473|
|STL|1.8263822546604833|
|ONT|1.7722329347353263|
|SAT|1.5303551318597992|
|OAK|1.4886721798643956|
|SMF|1.4779258223049847|
|LAX|1.4134227909403223|
|SLC| 1.362280151671023|
|JAX|1.2637766842227056|
+---+------------------+
only showing top 20 rows



## Componentes conexas

En nuestor grafo es posible llegar a cualquier aeropuerto desde cualquier otro, y por tanto solo hay una componente conexa y la columna `component` (identificador de la componente) tiene el mismo valor para todos los vértices.

In [9]:
spark.sparkContext.setCheckpointDir("/home/bigdata/data")
conCompResult = graph.connectedComponents(checkpointInterval=10)
conCompResult.show(100)

+---+-----------+
| id|  component|
+---+-----------+
|MSY|17179869184|
|GEG|17179869184|
|BUR|17179869184|
|SNA|17179869184|
|PVD|17179869184|
|OAK|17179869184|
|ORF|17179869184|
|CRW|17179869184|
|CMH|17179869184|
|IAH|17179869184|
|SJC|17179869184|
|BUF|17179869184|
|AUS|17179869184|
|LGB|17179869184|
|BFL|17179869184|
|RNO|17179869184|
|CHS|17179869184|
|HRL|17179869184|
|RSW|17179869184|
|TUL|17179869184|
|AMA|17179869184|
|ISP|17179869184|
|MAF|17179869184|
|EWR|17179869184|
|LAS|17179869184|
|JAN|17179869184|
|DEN|17179869184|
|ALB|17179869184|
|BOI|17179869184|
|IAD|17179869184|
|SEA|17179869184|
|MCI|17179869184|
|BNA|17179869184|
|CLT|17179869184|
|ABQ|17179869184|
|PBI|17179869184|
|SDF|17179869184|
|BDL|17179869184|
|DAL|17179869184|
|MRY|17179869184|
|CLE|17179869184|
|PDX|17179869184|
|BWI|17179869184|
|TPA|17179869184|
|OKC|17179869184|
|SMF|17179869184|
|PHX|17179869184|
|STL|17179869184|
|MHT|17179869184|
|LBB|17179869184|
|CRP|17179869184|
|SFO|17179869184|
|ELP|17179

## Consultas por estructura: rutas de vuelo entre aeropuertos sin conexión directa

Estamos indicando que queremos encontrar vértices **a**, **b** y **c** para que haya una arista de **a** a **b**, otra de **b** a **c**, pero no de **a** a **c**, por lo tanto, **a** y **c** no están conectados en un solo salto sino que requieren al menos dos. La restricción adicional evita que **a** y **c** sean el mismo vértice, ya que ningún aeropuerto está conectado consigo mismo, por lo que cada aeropuerto cumpliría individualmente la condición respecto a sí mismo.

In [10]:
res = graph\
 .find("(a)-[]->(b); (b)-[]->(c); !(a)-[]->(c)")\
 .filter("c.id !=a.id")

res.show()

+-----+-----+-----+
|    a|    b|    c|
+-----+-----+-----+
|[ABQ]|[LAX]|[BNA]|
|[AMA]|[LAS]|[TUL]|
|[AUS]|[MCI]|[PDX]|
|[AUS]|[MDW]|[PVD]|
|[AUS]|[PHX]|[STL]|
|[BDL]|[MDW]|[DTW]|
|[BHM]|[LAS]|[ELP]|
|[BHM]|[MCO]|[ORF]|
|[BNA]|[TPA]|[PBI]|
|[BUF]|[LAS]|[MAF]|
|[BUF]|[PHX]|[SNA]|
|[BUR]|[PHX]|[OMA]|
|[BUR]|[PHX]|[STL]|
|[CLE]|[BWI]|[SAT]|
|[CLE]|[MCO]|[ORF]|
|[DAL]|[ABQ]|[MDW]|
|[DAL]|[BHM]|[RDU]|
|[DEN]|[PHX]|[SNA]|
|[DTW]|[MDW]|[LIT]|
|[DTW]|[STL]|[BHM]|
+-----+-----+-----+
only showing top 20 rows



## Caminos más cortos: Breadth-first search

Vamos a encontrar el camino más corto entre dos aeropuertos que no están directamente conectados

<div class="alert alert-danger">
    <b>IMPORTANTE</b>: BFS (Breadth-first search) en Spark calcula el camino mínimo en términos de <b>número de saltos</b> entre dos vértices. No tiene en cuenta el peso de las aristas. Se podría implementar pero de forma personalizada, no con la función bfs().
</div>

Como hay varios caminos que tienen 2 saltos entre ABQ y BNA, la función `bfs` devuelve todos ellos.

In [11]:
paths = graph.bfs(fromExpr = "id = 'ABQ'", toExpr= "id = 'BNA'")
paths.show()

+-----+----------------+-----+----------------+-----+
| from|              e0|   v1|              e1|   to|
+-----+----------------+-----+----------------+-----+
|[ABQ]| [ABQ, OAK, 889]|[OAK]|[OAK, BNA, 1959]|[BNA]|
|[ABQ]| [ABQ, IAH, 744]|[IAH]| [IAH, BNA, 657]|[BNA]|
|[ABQ]| [ABQ, AUS, 619]|[AUS]| [AUS, BNA, 756]|[BNA]|
|[ABQ]| [ABQ, LAS, 487]|[LAS]|[LAS, BNA, 1588]|[BNA]|
|[ABQ]| [ABQ, DEN, 349]|[DEN]|[DEN, BNA, 1013]|[BNA]|
|[ABQ]|[ABQ, SEA, 1180]|[SEA]|[SEA, BNA, 1977]|[BNA]|
|[ABQ]| [ABQ, MCI, 718]|[MCI]| [MCI, BNA, 491]|[BNA]|
|[ABQ]|[ABQ, BWI, 1670]|[BWI]| [BWI, BNA, 588]|[BNA]|
|[ABQ]|[ABQ, TPA, 1497]|[TPA]| [TPA, BNA, 612]|[BNA]|
|[ABQ]| [ABQ, PHX, 328]|[PHX]|[PHX, BNA, 1448]|[BNA]|
|[ABQ]|[ABQ, MDW, 1121]|[MDW]| [MDW, BNA, 395]|[BNA]|
|[ABQ]| [ABQ, HOU, 759]|[HOU]| [HOU, BNA, 670]|[BNA]|
|[ABQ]| [ABQ, ONT, 631]|[ONT]|[ONT, BNA, 1751]|[BNA]|
|[ABQ]| [ABQ, LAX, 677]|[LAX]|[LAX, BNA, 1797]|[BNA]|
|[ABQ]|[ABQ, MCO, 1552]|[MCO]| [MCO, BNA, 616]|[BNA]|
|[ABQ]| [ABQ, SAN, 628]|[SAN